In [25]:
import requests
import zipfile
import io
import pandas as pd

# Definir URLs
dataRepo = 'https://github.com/Camilo0702/PC3-Bernabe/raw/main/appendData.zip'
countryCodesURL = 'https://github.com/Camilo0702/PC3-Bernabe/raw/main/countryCODES.xlsx'
fallecidosURL = 'https://github.com/Camilo0702/PC3-Bernabe/raw/main/fallecidosCOVID.csv'

# Descargar el archivo ZIP
response = requests.get(dataRepo)
country_codes_response = requests.get(countryCodesURL)
fallecidos_response = requests.get(fallecidosURL)

if response.status_code == 200 and country_codes_response.status_code == 200 and fallecidos_response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        allDFs = []
        print("Archivos en el ZIP:", zip_ref.namelist())

        for file_name in zip_ref.namelist():
            if file_name.startswith('__MACOSX/') or file_name.startswith('.'):
                continue
            
            if file_name.endswith('.xlsx') or file_name.endswith('.xls'):
                try:
                    with zip_ref.open(file_name) as file:
                        if file_name == 'appendData/level1.xlsx':
                            df = pd.read_excel(file, engine='openpyxl', header=3)
                        else:
                            df = pd.read_excel(file, engine='openpyxl')

                        df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_', regex=True)
                        df.columns = df.columns.str.replace(r'[^a-z0-9_]', '', regex=True)
                        df = df.fillna('')

                        print(f"Leído {file_name} con {len(df)} filas.")
                        print(df.head())
                        
                        allDFs.append(df)
                except Exception as e:
                    print(f"Error al procesar el archivo {file_name}: {e}")

    # Filtrar columnas deseadas
    columns_to_keep = ['hdi_rank', 'country', 'human_development_index_hdi', 
                       'life_expectancy_at_birth', 'expected_years_of_schooling', 
                       'mean_years_of_schooling', 'gross_national_income_gni_per_capita']
    
    allDFs_filtered = []
    for df in allDFs:
        filtered_df = df.loc[:, df.columns[df.columns.isin(columns_to_keep)]]
        filtered_df = filtered_df.fillna('')
        if not filtered_df.empty:
            allDFs_filtered.append(filtered_df)

    # Concatenar DataFrames
    if allDFs_filtered:
        allDFsConcat = pd.concat(allDFs_filtered, ignore_index=True, sort=False)
        print("DataFrame concatenado con éxito.")
        
        # Renombrar columnas
        allDFsConcat.columns = allDFsConcat.columns.str.strip().str.lower().str.replace(' ', '_', regex=True)

        # Guardar el resultado
        allDFsConcat.to_csv("Fragility.csv", index=False)
        print("El archivo Fragility.csv ha sido guardado correctamente.")
        
        # Guardar el archivo countryCODES.xlsx
        with open("countryCODES.xlsx", 'wb') as f:
            f.write(country_codes_response.content)
        print("El archivo countryCODES.xlsx ha sido guardado correctamente.")

        # Leer el archivo guardado
        country_codes = pd.read_excel("countryCODES.xlsx")

        # Limpiar nombres de países
        allDFsConcat['country'] = allDFsConcat['country'].str.strip().str.title().str.replace(r' \(.+\)', '', regex=True)
        country_codes['Country'] = country_codes['Country'].str.strip().str.title().str.replace(r' \(.+\)', '', regex=True)

        # Verificar nombres únicos de países en ambos DataFrames
        print("Nombres de países en allDFsConcat:", allDFsConcat['country'].unique())
        print("Nombres de países en country_codes:", country_codes['Country'].unique())

        # Realizar el merge
        merged_df = pd.merge(allDFsConcat, country_codes, how='left', left_on='country', right_on='Country')

        # Guardar el DataFrame fusionado
        merged_df.to_csv("Merged_Fragility.csv", index=False)
        print("El archivo Merged_Fragility.csv ha sido guardado correctamente.")

        # Procesar fallecidosCOVID.csv
        fallecidos_df = pd.read_csv(io.BytesIO(fallecidos_response.content), sep=';')

        # Imprimir los nombres de las columnas para verificar
        print("Columnas en fallecidos_df:", fallecidos_df.columns)

        # Ajustar los nombres de las columnas
        nombre_columna_fecha_fallecimiento = 'FECHA_FALLECIMIENTO'
        tipo_muerte_columna = 'CLASIFICACION_DEF'  # Cambia esto al nombre de la columna que contiene el tipo de muerte

        # Convertir la columna de fechas a datetime
        fallecidos_df[nombre_columna_fecha_fallecimiento] = pd.to_datetime(fallecidos_df[nombre_columna_fecha_fallecimiento], errors='coerce')

        # Agrupar por mes y clasificar
        reporte_mensual = fallecidos_df.groupby(fallecidos_df[nombre_columna_fecha_fallecimiento].dt.to_period('M')).size().reset_index(name='total_fallecidos')
        reporte_mensual['FECHA_FALLECIMIENTO'] = reporte_mensual['FECHA_FALLECIMIENTO'].dt.to_timestamp()
        reporte_mensual.to_csv("reporte_mensual_fallecidos.csv", index=False)
        print("El reporte mensual de fallecidos ha sido guardado como 'reporte_mensual_fallecidos.csv'.")

        # Reporte de fallecidos por tipo de muerte
        reporte_tipo_muerte = fallecidos_df.groupby([fallecidos_df[nombre_columna_fecha_fallecimiento].dt.to_period('M'), tipo_muerte_columna]).size().unstack(fill_value=0)
        reporte_tipo_muerte.index = reporte_tipo_muerte.index.to_timestamp()
        reporte_tipo_muerte.to_csv("reporte_fallecidos_tipo_muerte_mensual.csv")
        print("El reporte mensual de fallecidos por tipo de muerte ha sido guardado como 'reporte_fallecidos_tipo_muerte_mensual.csv'.")

        # Mostrar las tablas generadas
        print("\nReporte Mensual de Fallecidos:")
        print(reporte_mensual.head(), "\n")

        print("Reporte de Fallecidos por Tipo de Muerte:")
        print(reporte_tipo_muerte.head(), "\n")

else:
    if response.status_code != 200:
        print("Error al descargar el archivo:", response.status_code)
    if country_codes_response.status_code != 200:
        print("Error al descargar el archivo countryCODES.xlsx:", country_codes_response.status_code)
    if fallecidos_response.status_code != 200:
        print("Error al descargar el archivo fallecidosCOVID.csv:", fallecidos_response.status_code)

Archivos en el ZIP: ['appendData/', 'appendData/level1.xlsx', '__MACOSX/appendData/._level1.xlsx', 'appendData/.DS_Store', '__MACOSX/appendData/._.DS_Store', 'appendData/level4.xlsx', '__MACOSX/appendData/._level4.xlsx', 'appendData/level3.xlsx', '__MACOSX/appendData/._level3.xlsx', 'appendData/level2.xlsx', '__MACOSX/appendData/._level2.xlsx']
Leído appendData/level1.xlsx con 69 filas.
  unnamed_0    unnamed_1                       unnamed_2 unnamed_3  \
0  HDI rank      Country  Human Development Index (HDI)              
1                                                  Value             
2                                                   2021             
3         1  Switzerland                           0.962             
4         2       Norway                           0.961             

                  unnamed_4 unnamed_5                    unnamed_6 unnamed_7  \
0  Life expectancy at birth            Expected years of schooling             
1                   (years)  